In [ ]:
import geopandas as gpd
import numpy as np
import plotly.express as px
import plotly.offline as po
from keplergl import KeplerGl
from shapely.geometry import MultiPolygon, Point, Polygon, box

from srai.regionizers import VoronoiRegionizer

Tesellate whole Earth

In [ ]:
# 6 poles of the Earth
seeds_gdf = gpd.GeoDataFrame(
    {
        "geometry": [
            Point(0, 0),
            Point(90, 0),
            Point(180, 0),
            Point(-90, 0),
            Point(0, 90),
            Point(0, -90),
        ]
    },
    index=[1, 2, 3, 4, 5, 6],
    crs="EPSG:4326",
)

# Bounding box of the whole Earth
bbox = box(minx=-180, maxx=180, miny=-90, maxy=90)
bbox_gdf = gpd.GeoDataFrame({"geometry": [bbox]}, crs="EPSG:4326")

In [ ]:
seeds_gdf.plot()

In [ ]:
vr = VoronoiRegionizer(seeds=seeds_gdf)

In [ ]:
result_gdf = vr.transform(gdf=bbox_gdf)

In [ ]:
result_gdf

In [ ]:
fig = px.choropleth(
    result_gdf,
    geojson=result_gdf.geometry,
    locations=result_gdf.index,
    color=result_gdf.index,
    color_continuous_scale=px.colors.sequential.Viridis,
)
fig2 = px.scatter_geo(seeds_gdf, lat=seeds_gdf.geometry.y, lon=seeds_gdf.geometry.x)
fig.update_traces(marker={"opacity": 0.6}, selector=dict(type="choropleth"))
fig.add_trace(fig2.data[0])
fig.update_traces(marker_color="white", marker_size=10, selector=dict(type="scattergeo"))
fig.update_layout(coloraxis_showscale=False)
fig.update_geos(
    projection_type="orthographic", projection_rotation_lon=20, projection_rotation_lat=30
)
fig.update_layout(height=500, margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show(renderer="png")  # replace with fig.show() to allow interactivity

Tesellate single country

In [ ]:
world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
area = world[world.name == "United Kingdom"]

area = area.to_crs(epsg=4326)  # convert to wgs84
area_shape = area.iloc[0].geometry  # get the Polygon

In [ ]:
area

In [ ]:
N_POINTS = 100
# generate some random points within the bounds
minx, miny, maxx, maxy = area_shape.bounds

randx = np.random.uniform(minx, maxx, N_POINTS)
randy = np.random.uniform(miny, maxy, N_POINTS)
coords = np.vstack((randx, randy)).T

# use only the points inside the geographic area

pts = [p for p in list(map(Point, coords)) if p.within(area_shape)]

uk_seeds_gdf = gpd.GeoDataFrame(
    {"geometry": pts},
    index=list(range(len(pts))),
    crs="EPSG:4326",
)

del coords  # not used any more

In [ ]:
uk_seeds_gdf.plot()

In [ ]:
vr_uk = VoronoiRegionizer(seeds=uk_seeds_gdf)

In [ ]:
uk_result_gdf = vr_uk.transform(gdf=area)

In [ ]:
uk_result_gdf.head()

In [ ]:
fig = px.choropleth(
    uk_result_gdf,
    geojson=uk_result_gdf.geometry,
    locations=uk_result_gdf.index,
    color=uk_result_gdf.index,
    color_continuous_scale=px.colors.qualitative.Plotly,
)
fig2 = px.scatter_geo(uk_seeds_gdf, lat=uk_seeds_gdf.geometry.y, lon=uk_seeds_gdf.geometry.x)
fig.update_traces(marker={"opacity": 0.6}, selector=dict(type="choropleth"))
fig.add_trace(fig2.data[0])
fig.update_traces(marker_color="black", marker_size=6, selector=dict(type="scattergeo"))
fig.update_layout(coloraxis_showscale=False)
fig.update_geos(
    projection_type="natural earth",
    lataxis_range=[miny - 1, maxy + 1],
    lonaxis_range=[minx - 1, maxx + 1],
)
fig.update_layout(height=500, margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show(renderer="png")  # replace with fig.show() to allow interactivity

Difference between spherical voronoi and 2d voronoi

In [ ]:
"""
Geovoronoi package allows for a quick tesellation of the Earth using list of seeds on a projected 2d plane.
This results in straight lines with angles distorted and polygons differences
might be substantial during comparisons or any area calculations.
"""
# geovoronoi package isn't used in this package, but is optional and can be installed using pip install srai[docs]
from geovoronoi import voronoi_regions_from_coords

from shapely.geometry.polygon import orient
from plotly.subplots import make_subplots

In [ ]:
world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
area = world[world.name == "Poland"]

area = area.to_crs(epsg=4326)  # convert to wgs84
area_shape = area.iloc[0].geometry  # get the Polygon

In [ ]:
N_POINTS = 100
# generate some random points within the bounds
minx, miny, maxx, maxy = area_shape.bounds

randx = np.random.uniform(minx, maxx, N_POINTS)
randy = np.random.uniform(miny, maxy, N_POINTS)
coords = np.vstack((randx, randy)).T

# use only the points inside the geographic area

pts = [p for p in list(map(Point, coords)) if p.within(area_shape)]

pl_seeds_gdf = gpd.GeoDataFrame(
    {"geometry": pts},
    index=list(range(len(pts))),
    crs="EPSG:4326",
)

del coords

In [ ]:
region_polys, region_pts, unassigned_pts = voronoi_regions_from_coords(
    pts, area_shape, return_unassigned_points=True, per_geom=False
)

In [ ]:
def orient_geom(geom):
    if type(geom) == Polygon:
        return orient(geom, sign=-1)
    elif type(geom) == MultiPolygon:
        return MultiPolygon([orient(g, sign=-1) for g in geom.geoms])


pl_regions_2d_gdf = gpd.GeoDataFrame(
    {"geometry": [orient_geom(geom) for geom in region_polys.values()]},
    index=list(range(len(region_polys))),
    crs="EPSG:4326",
)

In [ ]:
pl_regions_2d_gdf

In [ ]:
vr_pl = VoronoiRegionizer(seeds=pl_seeds_gdf)

In [ ]:
pl_result_gdf = vr_pl.transform(gdf=area)

In [ ]:
pl_result_gdf

In [ ]:
choropleth_1 = px.choropleth(
    pl_result_gdf,
    geojson=pl_result_gdf.geometry,
    locations=pl_result_gdf.index,
    color=pl_result_gdf.index,
    color_continuous_scale=px.colors.qualitative.Plotly,
)

choropleth_2 = px.choropleth(
    pl_regions_2d_gdf,
    geojson=pl_regions_2d_gdf.geometry,
    locations=pl_regions_2d_gdf.index,
    color=pl_regions_2d_gdf.index,
    color_continuous_scale=px.colors.qualitative.Plotly,
)

points_plot = px.scatter_geo(pl_seeds_gdf, lat=pl_seeds_gdf.geometry.y, lon=pl_seeds_gdf.geometry.x)

fig = make_subplots(
    rows=2,
    cols=2,
    specs=[
        [{"type": "scattergeo"}, {"type": "scattergeo"}],
        [{"type": "scattergeo"}, {"type": "scattergeo"}],
    ],
    horizontal_spacing=0.01,
    vertical_spacing=0.0,
)

fig.add_trace(choropleth_1["data"][0], row=1, col=1)
fig.add_trace(choropleth_1["data"][0], row=2, col=1)
fig.add_trace(choropleth_2["data"][0], row=1, col=2)
fig.add_trace(choropleth_2["data"][0], row=2, col=2)
fig.add_trace(points_plot.data[0], row=1, col=1)
fig.add_trace(points_plot.data[0], row=1, col=2)
fig.add_trace(points_plot.data[0], row=2, col=1)
fig.add_trace(points_plot.data[0], row=2, col=2)

fig.update_traces(marker_color="black", marker_size=6, selector=dict(type="scattergeo"))
fig.update_layout(coloraxis_showscale=False)
fig.update_geos(
    projection_type="natural earth",
    lataxis_range=[miny - 1, maxy + 1],
    lonaxis_range=[minx - 1, maxx + 1],
    row=1,
)

fig.update_geos(
    projection_type="natural earth",
    lataxis_range=[miny + 1, maxy - 1],
    lonaxis_range=[minx + 2, maxx - 2],
    row=2,
)

fig.update_traces(marker={"opacity": 0.6}, selector=dict(type="choropleth"), row=1)
fig.update_traces(marker={"opacity": 0.25}, selector=dict(type="choropleth"), row=2)

fig.update_layout(
    height=800,
    width=800,
    title_text="Side By Side Subplots (Left: Spherical voronoi, Right: 2D voronoi)",
    margin={"r": 5, "t": 50, "l": 5, "b": 0},
)
fig.show(renderer="png")  # replace with fig.show() to allow interactivity